In [1]:
import os, math
import numpy as np
import pandas as pd
from datetime import datetime
from pprint import pprint
import pickle
import omero
import json

import ezomero
#   https://thejacksonlaboratory.github.io/ezomero/ezomero.html#module-ezomero

In [ ]:
fh="OVCA_TMA22_metadata.tsv"
metadf = pd.read_csv(fh, sep='\t', low_memory=False)

In [45]:
# Settings for OMERO
home = str(Path.home())
config = configparser.RawConfigParser()
config.read(r'{}\Documents\myKeyVariables.cfg'.format(home))   
envVars = dict(config.items('OMERO'))

# Settings for OMERO
user = envVars['user']
password = envVars['pass']
host = envVars['url']
port = 4064 #4080
secure = True #False

conn = ezomero.connect(user, password, 'Goode.E_Lab', host=host, port=port, secure=secure)
ezomero.print_projects(conn)

Projects:
	OVCA_TMA22:	408


In [46]:
ezomero.print_datasets(conn, project=408)
setDs = 712

Datasets in Project "OVCA_TMA22":
	INCell_MxIF:	712
	WSI_czi:	823


In [47]:
getImageByName = {}
imgs = ezomero.get_image_ids(conn, dataset=setDs)
for im in imgs:
    imgDesc, pix = ezomero.get_image(conn, im, xyzct=False, no_pixels=True)
    imName = imgDesc.getName().replace(".ome.tiff","")
    getImageByName[imName] = im
getImageByName

{'OVCA_TMA22_region_018': 6455,
 'OVCA_TMA22_region_116': 6456,
 'OVCA_TMA22_region_214': 6457,
 'OVCA_TMA22_region_110': 6458,
 'OVCA_TMA22_region_303': 6459,
 'OVCA_TMA22_region_107': 6460,
 'OVCA_TMA22_region_205': 6461,
 'OVCA_TMA22_region_312': 6462,
 'OVCA_TMA22_region_125': 6463,
 'OVCA_TMA22_region_064': 6464,
 'OVCA_TMA22_region_134': 6465,
 'OVCA_TMA22_region_223': 6466,
 'OVCA_TMA22_region_027': 6467,
 'OVCA_TMA22_region_036': 6468,
 'OVCA_TMA22_region_045': 6469,
 'OVCA_TMA22_region_242': 6470,
 'OVCA_TMA22_region_299': 6471,
 'OVCA_TMA22_region_055': 6472,
 'OVCA_TMA22_region_153': 6473,
 'OVCA_TMA22_region_345': 6474,
 'OVCA_TMA22_region_100': 6475,
 'OVCA_TMA22_region_224': 6476,
 'OVCA_TMA22_region_265': 6477,
 'OVCA_TMA22_region_135': 6478,
 'OVCA_TMA22_region_017': 6479,
 'OVCA_TMA22_region_083': 6480,
 'OVCA_TMA22_region_046': 6481,
 'OVCA_TMA22_region_331': 6482,
 'OVCA_TMA22_region_106': 6483,
 'OVCA_TMA22_region_319': 6484,
 'OVCA_TMA22_region_261': 6485,
 'OVCA_T

In [48]:
### Handy object to make annotations appear in native layer
nsStr = omero.constants.metadata.NSCLIENTMAPANNOTATION

for index, tmp in metadf.iterrows():
    if math.isnan(tmp['DeepCellCount']):
        continue
    formatTmp = { "Row x Column" : "{}x{}".format(tmp['Row'],tmp['Column']),
             "CellCount": tmp['DeepCellCount'],
             "Category":tmp['category'],
             "Malignancy":tmp['malignancytype'],
             "Histology":tmp['histology'],
             "Stage":tmp['stage'],
             "Grade":tmp['grade']
    }
    if not math.isnan(tmp['Participant']):
        formatTmp["StudyID"] = str(int(tmp['Participant']))
        
    if math.isnan(tmp['CD8TilScore']):
        formatTmp["TIL Score (CD8)"] = "?"
    else :
        formatTmp["TIL Score (CD8)"] = int(tmp['CD8TilScore'])
                  
    #print(formatTmp)
    _id = getImageByName[tmp['Sample']]
    ezomero.post_map_annotation(conn, object_id=_id, object_type="Image", kv_dict=formatTmp, ns=nsStr)
